In [2]:
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 2

import numpy as np
from equation.burgers import Burgers1D, InitialCondition, BoundaryCondition
import torch
from equation.derivative import diff

In [3]:
eq = Burgers1D(nu=1)
ic = InitialCondition()
bc = BoundaryCondition()

In [65]:
import torch.nn as nn
x = torch.tensor([[1.,2.],[3,4]], requires_grad=True)
u = nn.Linear(2, 1)(x)

In [68]:
u_t = torch.autograd.grad(
        u, x, grad_outputs=torch.ones_like(u), create_graph=True, allow_unused=True
    )[0]

In [69]:
u_t

tensor([[-0.6731,  0.5279],
        [-0.6731,  0.5279]], grad_fn=<MmBackward0>)

In [35]:
input = {"interior": torch.ones(10, 2), "initial": torch.ones(10, 2)*2, "boundary": torch.ones(10, 2)}
input = {k: v.requires_grad_(True) for k, v in input.items()}

In [15]:
def func(input):
    return torch.sum(input**2, axis=-1)

In [36]:
u = {k: func(v) for k, v in input.items()}

In [37]:
u["interior"].grad_fn.next_functions

((<PowBackward0 at 0x7f9cdc543730>, 0),)

In [32]:
input["interior"].grad_fn

In [38]:
t = input["interior"][:, 0]

In [39]:
u_t = diff(u["interior"], t)

In [40]:
print(u_t)

None


In [22]:
eq.residual(u, input)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<SelectBackward0>)
None


TypeError: ones_like(): argument 'input' (position 1) must be Tensor, not NoneType

In [7]:
ic.residual(u, t, x)

tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.], grad_fn=<SubBackward0>)

In [8]:
bc.residual(u, t, x)

tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.], grad_fn=<SubBackward0>)

In [9]:
eq.loss(u, t, x)

tensor(400., grad_fn=<MulBackward0>)

In [10]:
bc.loss(u, t, x)

AttributeError: 'BoundaryCondition' object has no attribute 'loss_weight'